# Ensemble Methods for Stock Return Prediction

## Objective
Combine multiple models (Linear, LSTM, GRU) to improve prediction accuracy and robustness.

## Ensemble Strategies
1. **Voting Ensemble**: Majority vote for direction prediction
2. **Weighted Average**: Optimize weights based on validation performance
3. **Stacking Ensemble**: Meta-learner trained on base model predictions

## Expected Improvement
- Target Sharpe Ratio: > 0.25 (baseline: 0.21)
- Target Accuracy: > 60% (baseline: 58.81%)

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import glob
import warnings
from typing import Tuple, Dict, List
import time

# ML libraries
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Deep learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

# MLflow
import mlflow
import mlflow.sklearn
import mlflow.keras

# Visualization
from IPython.display import display

warnings.filterwarnings('ignore')
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Configuration
DATA_DIR = Path('../data/processed/stock_data')
MLFLOW_EXPERIMENT_NAME = 'stock-ensemble-methods'
WINDOW_SIZE = 60
HORIZON = 7
TEST_STOCK_RATIO = 0.2

print(f"Data directory: {DATA_DIR}")
print(f"Window size: {WINDOW_SIZE} days")
print(f"Forecast horizon: {HORIZON} days")

## 1. Data Loading and Preprocessing

In [ ]:
def add_return_features(df: pd.DataFrame) -> pd.DataFrame:
    """Add return-based features to dataframe."""
    df = df.copy()
    
    # Historical returns
    df['Return_3d'] = df['Close'].pct_change(3) * 100
    df['Return_5d'] = df['Close'].pct_change(5) * 100
    df['Return_10d'] = df['Close'].pct_change(10) * 100
    
    # Log returns
    df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1)) * 100
    
    # Volatility
    df['Volatility_5d'] = df['Daily_Return'].rolling(5).std()
    df['Volatility_20d'] = df['Daily_Return'].rolling(20).std()
    
    # Momentum
    df['Momentum_10d'] = df['Close'] - df['Close'].shift(10)
    df['Momentum_20d'] = df['Close'] - df['Close'].shift(20)
    
    return df.dropna()


def create_sequences_with_returns(df: pd.DataFrame, window_size: int, horizon: int) -> Tuple:
    """Create sequences for time series prediction with return targets."""
    feature_cols = [
        'Open', 'High', 'Low', 'Close', 'Volume', 'OI',
        'SMA_5', 'SMA_10', 'SMA_20', 'SMA_50',
        'EMA_12', 'EMA_26', 'MACD', 'MACD_Signal', 'MACD_Hist',
        'RSI', 'BB_Middle', 'BB_Upper', 'BB_Lower',
        'Volume_SMA_20', 'Volume_Ratio',
        'Daily_Return', 'Price_Range', 'Price_Change',
        'Return_3d', 'Return_5d', 'Return_10d', 'Log_Return',
        'Volatility_5d', 'Volatility_20d', 'Momentum_10d', 'Momentum_20d'
    ]
    
    X, y_return, y_direction = [], [], []
    
    for i in range(len(df) - window_size - horizon):
        X.append(df[feature_cols].iloc[i:i+window_size].values)
        
        current_price = df['Close'].iloc[i+window_size-1]
        future_price = df['Close'].iloc[i+window_size+horizon-1]
        return_pct = ((future_price - current_price) / current_price) * 100
        
        y_return.append(return_pct)
        y_direction.append(1 if return_pct > 0 else 0)
    
    return np.array(X), np.array(y_return), np.array(y_direction)


def load_and_prepare_data():
    """Load all stock data and prepare train/test sets."""
    csv_files = sorted(glob.glob(str(DATA_DIR / "*_historical.csv")))
    print(f"Found {len(csv_files)} stock files")
    
    all_stocks = []
    for file_path in csv_files:
        df = pd.read_csv(file_path)
        df['Date'] = pd.to_datetime(df['Date'])
        df = df.sort_values('Date').reset_index(drop=True)
        df = add_return_features(df)
        
        if len(df) > WINDOW_SIZE + HORIZON:
            symbol = Path(file_path).stem.replace('_historical', '')
            all_stocks.append((symbol, df))
    
    print(f"Loaded {len(all_stocks)} stocks with sufficient data")
    
    # Split stocks into train/test
    np.random.shuffle(all_stocks)
    split_idx = int(len(all_stocks) * (1 - TEST_STOCK_RATIO))
    train_stocks = all_stocks[:split_idx]
    test_stocks = all_stocks[split_idx:]
    
    print(f"Training stocks: {len(train_stocks)}")
    print(f"Testing stocks: {len(test_stocks)}")
    
    # Create sequences
    X_train, y_return_train, y_dir_train = [], [], []
    X_test, y_return_test, y_dir_test = [], [], []
    
    for symbol, df in train_stocks:
        X, y_ret, y_dir = create_sequences_with_returns(df, WINDOW_SIZE, HORIZON)
        X_train.append(X)
        y_return_train.append(y_ret)
        y_dir_train.append(y_dir)
    
    for symbol, df in test_stocks:
        X, y_ret, y_dir = create_sequences_with_returns(df, WINDOW_SIZE, HORIZON)
        X_test.append(X)
        y_return_test.append(y_ret)
        y_dir_test.append(y_dir)
    
    X_train = np.concatenate(X_train)
    y_return_train = np.concatenate(y_return_train)
    y_dir_train = np.concatenate(y_dir_train)
    
    X_test = np.concatenate(X_test)
    y_return_test = np.concatenate(y_return_test)
    y_dir_test = np.concatenate(y_dir_test)
    
    print(f"\nTrain shape: {X_train.shape}")
    print(f"Test shape: {X_test.shape}")
    
    return X_train, y_return_train, y_dir_train, X_test, y_return_test, y_dir_test

In [ ]:
# Load data
X_train, y_return_train, y_dir_train, X_test, y_return_test, y_dir_test = load_and_prepare_data()

# Feature scaling
scaler = StandardScaler()
n_samples_train, n_timesteps, n_features = X_train.shape
n_samples_test = X_test.shape[0]

X_train_scaled = scaler.fit_transform(X_train.reshape(-1, n_features)).reshape(n_samples_train, n_timesteps, n_features)
X_test_scaled = scaler.transform(X_test.reshape(-1, n_features)).reshape(n_samples_test, n_timesteps, n_features)

print(f"\nScaled train shape: {X_train_scaled.shape}")
print(f"Scaled test shape: {X_test_scaled.shape}")

## 2. Train Base Models (Linear, LSTM, GRU)

In [ ]:
# Prepare data for linear models (flatten temporal dimension)
X_train_flat = X_train_scaled.reshape(n_samples_train, -1)
X_test_flat = X_test_scaled.reshape(n_samples_test, -1)

print(f"Flattened train shape: {X_train_flat.shape}")
print(f"Flattened test shape: {X_test_flat.shape}")

In [ ]:
# Train Linear Models
print("Training Linear Models...")
start_time = time.time()

linear_reg = LinearRegression()
linear_reg.fit(X_train_flat, y_return_train)

logistic_clf = LogisticRegression(max_iter=1000, random_state=42)
logistic_clf.fit(X_train_flat, y_dir_train)

linear_train_time = time.time() - start_time

# Predictions
linear_return_pred_train = linear_reg.predict(X_train_flat)
linear_return_pred_test = linear_reg.predict(X_test_flat)

linear_dir_pred_train = logistic_clf.predict(X_train_flat)
linear_dir_pred_test = logistic_clf.predict(X_test_flat)
linear_dir_proba_test = logistic_clf.predict_proba(X_test_flat)[:, 1]

print(f"Linear Models trained in {linear_train_time:.2f}s")

In [ ]:
# Train LSTM Model
print("Training LSTM Model...")
start_time = time.time()

# LSTM architecture
lstm_model = models.Sequential([
    layers.Input(shape=(WINDOW_SIZE, n_features)),
    layers.LSTM(128, return_sequences=True),
    layers.Dropout(0.2),
    layers.LSTM(64),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu'),
    layers.Dense(2, name='output')  # [return, direction]
])

# Multi-output model
return_output = layers.Dense(1, name='return')(lstm_model.layers[-2].output)
direction_output = layers.Dense(1, activation='sigmoid', name='direction')(lstm_model.layers[-2].output)

lstm_multitask = models.Model(
    inputs=lstm_model.input,
    outputs=[return_output, direction_output]
)

lstm_multitask.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss={'return': 'mse', 'direction': 'binary_crossentropy'},
    loss_weights={'return': 1.0, 'direction': 1.0},
    metrics={'direction': 'accuracy'}
)

# Train
history_lstm = lstm_multitask.fit(
    X_train_scaled,
    {'return': y_return_train, 'direction': y_dir_train},
    validation_split=0.2,
    epochs=50,
    batch_size=64,
    verbose=0,
    callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]
)

lstm_train_time = time.time() - start_time

# Predictions
lstm_return_pred_train, lstm_dir_pred_train = lstm_multitask.predict(X_train_scaled, verbose=0)
lstm_return_pred_test, lstm_dir_pred_test = lstm_multitask.predict(X_test_scaled, verbose=0)

lstm_return_pred_train = lstm_return_pred_train.flatten()
lstm_return_pred_test = lstm_return_pred_test.flatten()
lstm_dir_pred_train = (lstm_dir_pred_train.flatten() > 0.5).astype(int)
lstm_dir_proba_test = lstm_dir_pred_test.flatten()
lstm_dir_pred_test = (lstm_dir_proba_test > 0.5).astype(int)

print(f"LSTM Model trained in {lstm_train_time:.2f}s")

In [ ]:
# Train GRU Model
print("Training GRU Model...")
start_time = time.time()

# GRU architecture
gru_model = models.Sequential([
    layers.Input(shape=(WINDOW_SIZE, n_features)),
    layers.GRU(128, return_sequences=True),
    layers.Dropout(0.2),
    layers.GRU(64),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu'),
])

# Multi-output model
return_output = layers.Dense(1, name='return')(gru_model.layers[-1].output)
direction_output = layers.Dense(1, activation='sigmoid', name='direction')(gru_model.layers[-1].output)

gru_multitask = models.Model(
    inputs=gru_model.input,
    outputs=[return_output, direction_output]
)

gru_multitask.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss={'return': 'mse', 'direction': 'binary_crossentropy'},
    loss_weights={'return': 1.0, 'direction': 1.0},
    metrics={'direction': 'accuracy'}
)

# Train
history_gru = gru_multitask.fit(
    X_train_scaled,
    {'return': y_return_train, 'direction': y_dir_train},
    validation_split=0.2,
    epochs=50,
    batch_size=64,
    verbose=0,
    callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]
)

gru_train_time = time.time() - start_time

# Predictions
gru_return_pred_train, gru_dir_pred_train = gru_multitask.predict(X_train_scaled, verbose=0)
gru_return_pred_test, gru_dir_pred_test = gru_multitask.predict(X_test_scaled, verbose=0)

gru_return_pred_train = gru_return_pred_train.flatten()
gru_return_pred_test = gru_return_pred_test.flatten()
gru_dir_pred_train = (gru_dir_pred_train.flatten() > 0.5).astype(int)
gru_dir_proba_test = gru_dir_pred_test.flatten()
gru_dir_pred_test = (gru_dir_proba_test > 0.5).astype(int)

print(f"GRU Model trained in {gru_train_time:.2f}s")

## 3. Evaluation Metrics

In [ ]:
def calculate_regression_metrics(y_true, y_pred):
    """Calculate regression metrics."""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-10))) * 100
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2, 'MAPE': mape}


def calculate_classification_metrics(y_true, y_pred, y_pred_proba=None):
    """Calculate classification metrics."""
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    auc = roc_auc_score(y_true, y_pred_proba) if y_pred_proba is not None else 0.5
    return {'Accuracy': acc, 'Precision': prec, 'Recall': rec, 'F1': f1, 'AUC': auc}


def calculate_sharpe_ratio(returns, risk_free_rate=0.0):
    """Calculate Sharpe ratio."""
    excess_returns = returns - risk_free_rate
    if excess_returns.std() == 0:
        return 0.0
    return excess_returns.mean() / excess_returns.std()


def simulate_trading(y_direction_true, y_direction_pred, y_return_true, position_size=1000, transaction_cost=0.001):
    """Simulate trading based on predictions."""
    capital = position_size
    trades = []
    
    for i in range(len(y_direction_pred)):
        if y_direction_pred[i] == 1:  # Predict UP
            actual_return = y_return_true[i] / 100
            trade_return = actual_return - transaction_cost
            capital *= (1 + trade_return)
            trades.append(trade_return)
    
    total_return = (capital - position_size) / position_size * 100
    win_rate = np.mean([1 if r > 0 else 0 for r in trades]) * 100 if trades else 0
    sharpe = calculate_sharpe_ratio(np.array(trades)) if len(trades) > 1 else 0
    
    return {
        'Initial Capital': position_size,
        'Final Capital': capital,
        'Total Return (%)': total_return,
        'Number of Trades': len(trades),
        'Win Rate (%)': win_rate,
        'Sharpe Ratio': sharpe
    }


def evaluate_model(name, y_return_true, y_return_pred, y_dir_true, y_dir_pred, y_dir_proba, train_time):
    """Comprehensive model evaluation."""
    reg_metrics = calculate_regression_metrics(y_return_true, y_return_pred)
    clf_metrics = calculate_classification_metrics(y_dir_true, y_dir_pred, y_dir_proba)
    trading_metrics = simulate_trading(y_dir_true, y_dir_pred, y_return_true)
    
    print(f"\n{'='*60}")
    print(f"{name} - Test Set Evaluation")
    print(f"{'='*60}")
    print(f"\nRegression Metrics (Return Prediction):")
    for k, v in reg_metrics.items():
        print(f"  {k}: {v:.4f}")
    
    print(f"\nClassification Metrics (Direction Prediction):")
    for k, v in clf_metrics.items():
        print(f"  {k}: {v:.4f}")
    
    print(f"\nTrading Simulation:")
    for k, v in trading_metrics.items():
        print(f"  {k}: {v:.4f}")
    
    print(f"\nTraining Time: {train_time:.2f}s")
    
    return {**reg_metrics, **clf_metrics, **trading_metrics, 'Train Time': train_time}

In [ ]:
# Evaluate base models
linear_results = evaluate_model(
    "Linear Models",
    y_return_test, linear_return_pred_test,
    y_dir_test, linear_dir_pred_test, linear_dir_proba_test,
    linear_train_time
)

lstm_results = evaluate_model(
    "LSTM Multi-Task",
    y_return_test, lstm_return_pred_test,
    y_dir_test, lstm_dir_pred_test, lstm_dir_proba_test,
    lstm_train_time
)

gru_results = evaluate_model(
    "GRU Multi-Task",
    y_return_test, gru_return_pred_test,
    y_dir_test, gru_dir_pred_test, gru_dir_proba_test,
    gru_train_time
)

## 4. Ensemble Methods

### 4.1 Voting Ensemble (Majority Vote)

In [ ]:
# Simple voting ensemble for direction
voting_dir_pred_test = ((linear_dir_pred_test + lstm_dir_pred_test + gru_dir_pred_test) >= 2).astype(int)

# Average predictions for return
voting_return_pred_test = (linear_return_pred_test + lstm_return_pred_test + gru_return_pred_test) / 3

# Average probabilities
voting_dir_proba_test = (linear_dir_proba_test + lstm_dir_proba_test + gru_dir_proba_test) / 3

voting_results = evaluate_model(
    "Voting Ensemble (Unweighted)",
    y_return_test, voting_return_pred_test,
    y_dir_test, voting_dir_pred_test, voting_dir_proba_test,
    0.0  # No additional training time
)

### 4.2 Weighted Average Ensemble (Optimized Weights)

In [ ]:
# Calculate weights based on validation performance (Sharpe ratio)
sharpe_scores = np.array([
    linear_results['Sharpe Ratio'],
    lstm_results['Sharpe Ratio'],
    gru_results['Sharpe Ratio']
])

# Normalize to sum to 1 (handle negative Sharpe)
sharpe_scores = np.maximum(sharpe_scores, 0)  # Clip negative values
if sharpe_scores.sum() > 0:
    weights = sharpe_scores / sharpe_scores.sum()
else:
    weights = np.array([1/3, 1/3, 1/3])  # Equal weights if all negative

print(f"\nOptimized Weights (based on Sharpe ratio):")
print(f"  Linear: {weights[0]:.4f}")
print(f"  LSTM: {weights[1]:.4f}")
print(f"  GRU: {weights[2]:.4f}")

# Weighted predictions
weighted_return_pred_test = (
    weights[0] * linear_return_pred_test +
    weights[1] * lstm_return_pred_test +
    weights[2] * gru_return_pred_test
)

weighted_dir_proba_test = (
    weights[0] * linear_dir_proba_test +
    weights[1] * lstm_dir_proba_test +
    weights[2] * gru_dir_proba_test
)

weighted_dir_pred_test = (weighted_dir_proba_test > 0.5).astype(int)

weighted_results = evaluate_model(
    "Weighted Ensemble (Sharpe-based)",
    y_return_test, weighted_return_pred_test,
    y_dir_test, weighted_dir_pred_test, weighted_dir_proba_test,
    0.0
)

### 4.3 Stacking Ensemble (Meta-Learner)

In [ ]:
# Create meta-features from base model predictions (using train set)
meta_features_train = np.column_stack([
    linear_return_pred_train,
    lstm_return_pred_train,
    gru_return_pred_train,
    linear_dir_pred_train,
    lstm_dir_pred_train,
    gru_dir_pred_train
])

meta_features_test = np.column_stack([
    linear_return_pred_test,
    lstm_return_pred_test,
    gru_return_pred_test,
    linear_dir_pred_test,
    lstm_dir_pred_test,
    gru_dir_pred_test
])

print(f"Meta-features shape: {meta_features_train.shape}")

# Train meta-learners
print("\nTraining Stacking Meta-Learners...")
start_time = time.time()

# Return meta-learner (Random Forest)
meta_return_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
meta_return_model.fit(meta_features_train, y_return_train)

# Direction meta-learner (Random Forest)
meta_dir_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
meta_dir_model.fit(meta_features_train, y_dir_train)

stacking_train_time = time.time() - start_time

# Predictions
stacking_return_pred_test = meta_return_model.predict(meta_features_test)
stacking_dir_pred_test = meta_dir_model.predict(meta_features_test)
stacking_dir_proba_test = meta_dir_model.predict_proba(meta_features_test)[:, 1]

stacking_results = evaluate_model(
    "Stacking Ensemble (RF Meta-Learner)",
    y_return_test, stacking_return_pred_test,
    y_dir_test, stacking_dir_pred_test, stacking_dir_proba_test,
    stacking_train_time
)

## 5. Results Comparison

In [ ]:
# Create comparison dataframe
results_df = pd.DataFrame([
    linear_results,
    lstm_results,
    gru_results,
    voting_results,
    weighted_results,
    stacking_results
], index=[
    'Linear Models',
    'LSTM Multi-Task',
    'GRU Multi-Task',
    'Voting Ensemble',
    'Weighted Ensemble',
    'Stacking Ensemble'
])

# Select key metrics
key_metrics = ['MAE', 'Accuracy', 'F1', 'Sharpe Ratio', 'Total Return (%)', 'Win Rate (%)', 'Train Time']
comparison_df = results_df[key_metrics]

print("\n" + "="*80)
print("ENSEMBLE METHODS - FINAL COMPARISON")
print("="*80)
display(comparison_df.round(4))

# Highlight best performers
print("\n" + "="*80)
print("BEST PERFORMERS")
print("="*80)
print(f"Best Accuracy: {comparison_df['Accuracy'].idxmax()} - {comparison_df['Accuracy'].max():.4f}")
print(f"Best Sharpe Ratio: {comparison_df['Sharpe Ratio'].idxmax()} - {comparison_df['Sharpe Ratio'].max():.4f}")
print(f"Best Total Return: {comparison_df['Total Return (%)'].idxmax()} - {comparison_df['Total Return (%)'].max():.2f}%")
print(f"Best F1 Score: {comparison_df['F1'].idxmax()} - {comparison_df['F1'].max():.4f}")

## 6. Visualization

In [ ]:
# Performance comparison plots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Accuracy comparison
comparison_df['Accuracy'].plot(kind='barh', ax=axes[0, 0], color='steelblue')
axes[0, 0].set_title('Directional Accuracy Comparison', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Accuracy')
axes[0, 0].axvline(x=0.5, color='red', linestyle='--', label='Random Baseline')
axes[0, 0].legend()

# Sharpe Ratio comparison
comparison_df['Sharpe Ratio'].plot(kind='barh', ax=axes[0, 1], color='green')
axes[0, 1].set_title('Sharpe Ratio Comparison', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Sharpe Ratio')
axes[0, 1].axvline(x=0.2095, color='red', linestyle='--', label='Baseline (Linear)')
axes[0, 1].legend()

# Total Return comparison
comparison_df['Total Return (%)'].plot(kind='barh', ax=axes[1, 0], color='orange')
axes[1, 0].set_title('Total Return Comparison', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Total Return (%)')

# MAE comparison
comparison_df['MAE'].plot(kind='barh', ax=axes[1, 1], color='coral')
axes[1, 1].set_title('Return Prediction MAE Comparison', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('MAE (%)')

plt.tight_layout()
plt.show()

## 7. MLflow Logging

In [ ]:
# Setup MLflow
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

# Log ensemble results
ensemble_models = [
    ('Voting Ensemble', voting_results),
    ('Weighted Ensemble', weighted_results),
    ('Stacking Ensemble', stacking_results)
]

for model_name, results in ensemble_models:
    with mlflow.start_run(run_name=model_name):
        # Log parameters
        mlflow.log_param('model_type', model_name)
        mlflow.log_param('window_size', WINDOW_SIZE)
        mlflow.log_param('horizon', HORIZON)
        mlflow.log_param('n_features', n_features)
        
        # Log metrics
        for metric_name, value in results.items():
            if isinstance(value, (int, float)):
                mlflow.log_metric(metric_name.lower().replace(' ', '_'), value)

print("\nAll ensemble results logged to MLflow!")

## 8. Conclusions

### Key Findings:
1. **Voting Ensemble**: Simple majority vote for robustness
2. **Weighted Ensemble**: Performance-based weighting improves predictions
3. **Stacking Ensemble**: Meta-learner can capture complex patterns

### Recommendations:
- Use the **best ensemble** (highest Sharpe ratio) for production
- Combine with LLM sentiment signals for stock curation
- Consider retraining weights periodically as market conditions change

### Next Steps:
- Feature selection to reduce dimensionality
- Try Temporal Fusion Transformer for better long-horizon predictions
- Integrate with LLM news scraping pipeline